# <span style="color:teal">RBFE Network - Analysis</span>


In [1]:
# import libraries
import BioSimSpace as BSS
import os
import glob
import csv
import numpy as np
import math
import pandas as pd
import networkx as nx
import yaml
from scipy.stats import sem as sem
from scipy.stats import bootstrap
from sklearn.metrics import mean_absolute_error as mae
import pickle
import tempfile
import itertools
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd 
from rdkit import Chem

# try w cinnabar
sys.path.insert(1, '/home/anna/Documents/cinnabar')
import cinnabar
from cinnabar import wrangle,plotting
print(cinnabar.__file__)

try:
    import pipeline
except:
    print("adding code to the pythonpath...")
    code = '/home/anna/Documents/code/python'
    if code not in sys.path:
        sys.path.insert(1, code)
    import pipeline

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest 

/home/anna/Documents/cinnabar/cinnabar/__init__.py
adding code to the pythonpath...


In [2]:
# folders
protein = "tyk2"
file_ext = "MBAR_alchemlyb_benchmark" # for results files

# define all the folder locations
bench_folder = f"/home/anna/Documents/benchmark"
main_folder = f"{bench_folder}/{protein}_benchmark"
out_folder = f"{main_folder}/outputs"
res_folder = f"../test/results"
temp_folder = f"{main_folder}/outputs/results/temp"
exp_folder = f"{bench_folder}/inputs/experimental"

# make folders that may not exist
folder_list = [res_folder, temp_folder]
for fold in folder_list:
    validate.folder_path(fold, create=True)

# files
net_file = f"{main_folder}/execution_model/network_lomap.dat"
exp_file = f"{exp_folder}/{protein}.yml"
exp_file_dat = f"{res_folder}/exp_data_{protein}.dat"

# OUTPUT
file_ext_out = "test" # for how files will be written

# files that will get written
comp_pert_file_name = f"computed_perturbations_average_{file_ext_out}"
cinnabar_file = f"cinnabar_format_{file_ext_out}"
# TODO add file names for saving graphs

In [9]:
res_dir = "/home/anna/Documents/benchmark/tyk2_benchmark/outputs"
res_obj = analysis_engines(res_dir, net_file=net_file, exp_file=exp_file, engines="SOMD")
res_obj.compute()
res_obj.calc_val_dict
# res_obj._results_repeat_files["SOMD"]

engine input not recognised. Will use all engines.
no results output folder provided, writing all output to the 'results_directory'.


{'AMBER': {'lig_ejm54': (-1.0428697384819718, 0.023611936309945885),
  'lig_ejm55': (0.11953285550274204, 0.013442979584204524),
  'lig_ejm45': (1.5626085568721744, 0.018311612715938765),
  'lig_ejm53': (0.9625424313144026, 0.023072431759993003),
  'lig_ejm48': (-0.328095848662203, 0.023783984972893437),
  'lig_ejm44': (1.1321833683372324, 0.01431596983631065),
  'lig_ejm43': (-0.17763425082766435, 0.014231054538892556),
  'lig_ejm47': (-0.7031996975176686, 0.016808480174963454),
  'lig_ejm42': (0.29912802231729585, 0.009296954324983479),
  'lig_ejm46': (-0.5047490062497196, 0.01482169478479492),
  'lig_ejm50': (-0.2645545857759153, 0.042723858948738945),
  'lig_ejm31': (-0.1412410055351423, 0.012137397625099104),
  'lig_ejm49': (-0.5942134945497212, 0.02170950164168538),
  'lig_jmc27': (0.45652135463180343, 0.02108053890340404),
  'lig_jmc23': (-0.4950176982922253, 0.02515373756321302),
  'lig_jmc30': (-0.28094126308341605, 0.06758777248990212)},
 'SOMD': {'lig_ejm45': (0.662897151829

In [14]:
from cinnabar import wrangle,plotting
network = wrangle.FEMap(f"/home/anna/Documents/benchmark/tyk2_benchmark/outputs/cinnabar_SOMD.csv")

for node in network.graph.nodes(data=True):
    # print(node[1]["name"], node[1]["calc_DG"])
    # print(node[1])
    print(node[1]["name"], node[1]["exp_DG"])


{'lig_ejm45': (0.6628971518294626, 0.09591119801726128),
 'lig_ejm53': (1.769333311164411, 0.07158050409152511),
 'lig_ejm44': (1.5075937936610808, 0.053013995917800524),
 'lig_ejm54': (-1.1203935699722685, 0.08913646490103168),
 'lig_ejm55': (-0.15727890809445344, 0.060928173333954554),
 'lig_ejm48': (0.8540954923784945, 0.05972513580493125),
 'lig_ejm43': (0.107351523930189, 0.04952805603837424),
 'lig_ejm46': (-0.9354130580074901, 0.039141122026914124),
 'lig_ejm47': (-0.6718014300169751, 0.056094357961341886),
 'lig_ejm42': (0.6658810346903803, 0.02990246178691814),
 'lig_ejm50': (0.285896142398987, 0.049694388075367316),
 'lig_ejm49': (-0.07132776562176932, 0.15921881827387552),
 'lig_ejm31': (0.34351612540573934, 0.031003755988448754),
 'lig_jmc27': (-1.2881308743468858, 0.04849040129185547),
 'lig_jmc30': (-0.7868453203553116, 0.05751638976746995),
 'lig_jmc23': (-0.4211576796046013, 0.04280905865557756),
 'lig_jmc28': (-0.74421596943899, 0.045740597531330966)}

In [13]:
# import networkanalysis
# import experiments
# import stats

exp_dicts = res_obj._get_exp_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/')
for key in exp_dicts[0]:
    print(f"{key} : {exp_dicts[0][key][0]}")

free_fwf_dict = res_obj._get_ana_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/', engine="SOMD")
for key in free_fwf_dict:
    print(f"{key} : {free_fwf_dict[key][0]}")

fwf_stats = res_obj._get_stats_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/', engine="SOMD")


lig_ejm31 : 0.09554456211087381
lig_ejm42 : -0.14617801546097248
lig_ejm43 : 1.3886502840592203
lig_ejm44 : 2.239441725990767
lig_ejm45 : 0.08299333578780074
lig_ejm46 : -1.6903948748417696
lig_ejm47 : -0.05962612989820451
lig_ejm48 : 0.6418015718625703
lig_ejm49 : 1.9058204895277662
lig_ejm50 : 0.6661380623268188
lig_ejm53 : 1.7343155913659634
lig_ejm54 : -0.9024152875182272
lig_ejm55 : 0.43622104340109474
lig_jmc23 : -2.0792858601111703
lig_jmc27 : -1.6542528257313989
lig_jmc28 : -1.3497183935515489
lig_jmc30 : -1.3090552793195833
Added additional data to 44 edges; added 0 new edges.
Added additional data to 43 edges; added 0 new edges.
lig_ejm31 : 0.4438846195538745
lig_ejm42 : 0.5315348593421803
lig_ejm43 : 0.9263060947904159
lig_ejm44 : 1.6525224846333026
lig_ejm45 : 0.34699773961395514
lig_ejm46 : -0.6500995637874711
lig_ejm47 : -0.5060220552959447
lig_ejm48 : 0.28664870537773124
lig_ejm49 : -1.083365506415589
lig_ejm50 : -0.31127086389257824
lig_ejm53 : 2.1805905756579995
lig_ej

In [ ]:
# cycle closures
for eng in engines:

    pert_dict = values_dict[eng]["pert_results"]
    cycle_closures = # get from the network graph object

    cycles = pipeline.analysis.make_dict_cycle_closures(pert_dict, cycle_closures)    

    print(f"{eng} cycle vals is {cycles[1]}")
    print(f"{eng} cycle mean is {cycles[2]}")
    print(f"{eng} cycle deviation is {cycles[3]}")

    values_dict[eng]["cycle_closures"] = (cycles[0]) # the cycles dict

In [ ]:
from cinnabar import wrangle,plotting
network = wrangle.FEMap(f"{res_folder}/{cinnabar_file}_{eng}.csv")

In [ ]:
for eng in engines:
    convert_cinnabar_file(values_dict[eng]["results_files"], values_dict["experimental"]["val_results"], f"{res_folder}/{cinnabar_file}_{eng}")

    network = wrangle.FEMap(f"{res_folder}/{cinnabar_file}_{eng}.csv")
    # plot the perturbations
    plotting.plot_DDGs(network.graph, filename=f"{res_folder}/DDGs_{file_ext_out}_{eng}.png", title=f"DDGs_{protein}_{file_ext_out}_{eng}")
    #plot the ligands
    plotting.plot_DGs(network.graph, filename=f"{res_folder}/DGs_{file_ext_out}_{eng}.png", title=f"DGs_{protein}_{file_ext_out}_{eng}")